# 🔒 FraudGuard Training Notebook

**AD-RL-GNN Fraud Detection** | Full training pipeline with mini-batch processing

This notebook trains the FraudGuard model on the IEEE-CIS fraud detection dataset using:
- **NeighborLoader** for memory-efficient mini-batch training
- **FAISS** for similarity graph construction (GPU if available, CPU fallback)
- **CrossEntropyLoss** for class-imbalanced learning
- **MLflow** for experiment tracking and model versioning

## 1️⃣ Setup Environment

In [1]:
# Mount Google Drive for data storage
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Clone repository
!git clone https://github.com/govind104/fraudguard.git
%cd fraudguard

Cloning into 'fraudguard'...
remote: Enumerating objects: 246, done.
remote: Counting objects: 100% (246/246), done.
remote: Compressing objects: 100% (124/124), done.
remote: Total 246 (delta 116), reused 237 (delta 110), pack-reused 0 (from 0)
Receiving objects: 100% (246/246), 173.66 KiB | 6.95 MiB/s, done.
Resolving deltas: 100% (116/116), done.
/content/fraudguard


In [3]:
# Install dependencies
# Note: faiss-gpu may not be available on Python 3.12
# The code will fallback to faiss-cpu automatically
# GNN training STILL runs on GPU - only graph building uses CPU FAISS
!pip install -q torch torch-geometric pandas numpy scikit-learn pyyaml structlog mlflow

# Try faiss-gpu first, fallback to faiss-cpu
import subprocess
result = subprocess.run(['pip', 'install', '-q', 'faiss-gpu'], capture_output=True)
if result.returncode != 0:
    print('⚠️ faiss-gpu not available, using faiss-cpu')
    print('   (Graph building on CPU, but GNN training still runs on GPU!)')
    !pip install -q faiss-cpu
else:
    print('✓ faiss-gpu installed')

# Installing torch-scatter and torch-sparse for NeighborLoader
import torch

# 1. Get exact versions
pt_version = torch.__version__.split('+')[0]  # e.g., 2.5.1
cuda_version = "cu" + torch.version.cuda.replace('.', '')  # e.g., cu124
wheel_url = f"https://data.pyg.org/whl/torch-{pt_version}+{cuda_version}.html"

print(f"PyTorch: {pt_version}, CUDA: {cuda_version}")
print(f"Downloading from: {wheel_url}")

# 2. Install with visible output (force reinstall to fix broken partial installs)
!pip install --force-reinstall torch-scatter torch-sparse -f $wheel_url

# Install repo in editable mode
!pip install -e .

print('\n✓ Environment setup complete')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 135.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 103.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 71.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 787.9/787.9 kB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207

Obtaining file:///content/fraudguard
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.0/29.0 MB 86.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 123.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 95.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 272.8/272.8 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.0/66.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71


✓ Environment setup complete


In [1]:
try:
    import torch_scatter
    import torch_sparse
    import fraudguard
    print("✅ Success! Libraries are installed and loaded.")
except ImportError as e:
    print(f"❌ Still missing libraries: {e}")
    # Only if you see this error should you go back and install again.

✅ Success! Libraries are installed and loaded.


## 2️⃣ Configuration

In [2]:
import os
import gc
import sys
import time
import numpy as np
import faiss
import mlflow
import mlflow.pytorch

# Data paths - Point to your Google Drive folders
DATA_DIR = "/content/drive/MyDrive/ieee-fraud-detection"
MODELS_DIR = "/content/drive/MyDrive/fraudguard-models"
LOGS_DIR = "/content/drive/MyDrive/fraudguard-logs"

# Training parameters
SAMPLE_FRAC = 1.0           # Use full dataset (1.0 = 100%)
MAX_EPOCHS = 30
BATCH_SIZE = 2048           # Batch size for NeighborLoader
NUM_NEIGHBORS = [25, 10]    # 2-hop neighborhood sampling
LEARNING_RATE = 0.003       # Adam learning rate
FRAUD_WEIGHT = 25.0         # Class weight for fraud (minority class)
GRADIENT_CLIP = 1.0         # Max gradient norm

# MCD Alpha values for A/B comparison
BASELINE_ALPHA = 0.0        # No MCD for baseline
GOLD_ALPHA = 0.80           # Aggressive MCD for AD-RL-GNN

# Create directories
os.makedirs(MODELS_DIR, exist_ok=True)
os.makedirs(LOGS_DIR, exist_ok=True)

# Initialize MLflow
MLFLOW_DIR = f"{LOGS_DIR}/mlruns"
mlflow.set_tracking_uri(f"file://{MLFLOW_DIR}")
mlflow.set_experiment("FraudGuard-GNN")
print(f"\n📊 MLflow tracking: {MLFLOW_DIR}")

print(f"Data: {DATA_DIR}")
print(f"Models: {MODELS_DIR}")
print(f"Logs: {LOGS_DIR}")
print(f"\nBatch size: {BATCH_SIZE}")
print(f"Sample fraction: {SAMPLE_FRAC*100:.0f}%")

2026/01/22 13:07:04 INFO mlflow.tracking.fluent: Experiment with name 'FraudGuard-GNN' does not exist. Creating a new experiment.



📊 MLflow tracking: /content/drive/MyDrive/fraudguard-logs/mlruns
Data: /content/drive/MyDrive/ieee-fraud-detection
Models: /content/drive/MyDrive/fraudguard-models
Logs: /content/drive/MyDrive/fraudguard-logs

Batch size: 2048
Sample fraction: 100%


## 3️⃣ Verify GPU and FAISS

In [3]:
import torch

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    print("\n✓ GNN training will run on GPU")
else:
    print("\n⚠️ WARNING: No GPU detected. Go to Runtime > Change runtime type > GPU")

# Check FAISS GPU
faiss_gpus = faiss.get_num_gpus() if hasattr(faiss, 'get_num_gpus') else 0
print(f"\nFAISS GPUs: {faiss_gpus}")
if faiss_gpus == 0:
    print("   (Using CPU FAISS for graph building - this is OK)")

PyTorch version: 2.9.0+cu126
CUDA available: True
GPU: Tesla T4
VRAM: 15.8 GB

✓ GNN training will run on GPU

FAISS GPUs: 0
   (Using CPU FAISS for graph building - this is OK)


## 4️⃣ Load and Preprocess Data

In [4]:
sys.path.insert(0, '/content/fraudguard')

from pathlib import Path
from torch_geometric.data import Data
from torch_geometric.loader import NeighborLoader
from src.data.loader import FraudDataLoader
from src.utils.config import load_data_config, load_model_config
from src.utils.device_utils import set_seed, get_device
from src.training.trainer import FraudTrainer
from src.training.evaluator import Evaluator

set_seed(42)
device = get_device()
print(f"Using device: {device}")

# Load config and override path with notebook variable
data_cfg = load_data_config()
data_cfg.paths.raw_data_dir = Path(DATA_DIR)

# Load data with corrected path
loader = FraudDataLoader(config=data_cfg)
df = loader.load_train_data(sample_frac=SAMPLE_FRAC)
train_df, val_df, test_df = loader.create_splits(df)

print(f"\nData loaded:")
print(f"  Train: {len(train_df):,}")
print(f"  Val: {len(val_df):,}")
print(f"  Test: {len(test_df):,}")
print(f"  Fraud rate: {df['isFraud'].mean()*100:.2f}%")

# Initialize evaluator for metrics computation
evaluator = Evaluator()

Loading faiss with CPU support (no GPU detected).
Using device: cuda

Data loaded:
  Train: 354,324
  Val: 118,108
  Test: 118,108
  Fraud rate: 3.50%


## 5️⃣ Run Full AD-RL-GNN Pipeline

We use the `FraudTrainer` class to orchestrate the full pipeline, including:
1. **AdaptiveMCD**: Intelligent majority downsampling
2. **RL Agent**: Dynamic subgraph selection (Random Walk, K-Hop, K-Ego)
3. **Graph Enhancement**: Adding semantic edges
4. **GNN Training**: CrossEntropyLoss (25x weight)

# Vanilla Baseline GNN

In [5]:
start_time = time.time()

model_cfg = load_model_config()
data_cfg = load_data_config()
data_cfg.paths.raw_data_dir = Path(DATA_DIR)

# Aligned Hyperparameters (Strict Ceteris Paribus)
model_cfg.training["max_epochs"] = MAX_EPOCHS
model_cfg.training["learning_rate"] = LEARNING_RATE
model_cfg.adaptive_mcd["alpha"] = BASELINE_ALPHA  # No MCD
model_cfg.graph.similarity_threshold = 0.75

print("Initializing Vanilla Baseline (No MCD, No RL)...")
trainer = FraudTrainer(model_config=model_cfg, data_config=data_cfg, device=device)

# Preprocess
trainer._preprocess(train_df, val_df, test_df)
trainer._build_graph()
trainer._prepare_labels(train_df, val_df, test_df)

# Reset VRAM Monitor
torch.cuda.reset_peak_memory_stats()

# Apply class weight
weights = torch.tensor([1.0, FRAUD_WEIGHT]).to(device)
trainer._init_model()
trainer.criterion = torch.nn.CrossEntropyLoss(weight=weights)
model = trainer.model
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-4)

# Loaders
optimized_data = Data(x=trainer.X_full, edge_index=trainer.edge_index, y=trainer.all_labels)
optimized_data.train_mask = trainer.train_mask
optimized_data.val_mask = trainer.val_mask
train_loader = NeighborLoader(optimized_data, num_neighbors=NUM_NEIGHBORS, batch_size=BATCH_SIZE, input_nodes=optimized_data.train_mask, shuffle=True)
val_loader = NeighborLoader(optimized_data, num_neighbors=NUM_NEIGHBORS, batch_size=BATCH_SIZE, input_nodes=optimized_data.val_mask, shuffle=False)

print(f"\n Starting Baseline Training ({MAX_EPOCHS} Epochs)...")
best_gmeans_baseline = 0

# MLflow: Start baseline run
with mlflow.start_run(run_name="Vanilla-Baseline"):
    mlflow.log_params({
        "model_type": "baseline",
        "alpha": BASELINE_ALPHA,
        "fraud_weight": FRAUD_WEIGHT,
        "learning_rate": LEARNING_RATE,
        "batch_size": BATCH_SIZE,
        "max_epochs": MAX_EPOCHS,
        "num_neighbors": str(NUM_NEIGHBORS),
        "gradient_clip": GRADIENT_CLIP
    })

    for epoch in range(MAX_EPOCHS):
        model.train()
        for batch in train_loader:
            batch = batch.to(device)
            optimizer.zero_grad()
            out = model(batch.x, batch.edge_index)
            loss = trainer.criterion(out[:batch.batch_size], batch.y[:batch.batch_size])

            # Gradient clipping
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=GRADIENT_CLIP)
            optimizer.step()

        # Eval every epoch
        model.eval()
        all_preds, all_true = [], []
        with torch.no_grad():
            for batch in val_loader:
                batch = batch.to(device)
                out = model(batch.x, batch.edge_index)
                pred = out[:batch.batch_size].argmax(dim=1)
                all_preds.extend(pred.cpu().numpy())
                all_true.extend(batch.y[:batch.batch_size].cpu().numpy())

        metrics = evaluator.compute_metrics(np.array(all_true), np.array(all_preds))
        gmeans = metrics['gmeans']

        # MLflow: Log metrics per epoch
        mlflow.log_metrics({
            "gmeans": gmeans,
            "recall": metrics['recall'],
            "specificity": metrics['specificity']
        }, step=epoch)

        print(f"Baseline Epoch {epoch+1:>2} | Spec: {metrics['specificity']*100:.2f}% | Recall: {metrics['recall']*100:.2f}% | G-Means: {gmeans*100:.2f}%")

        if gmeans > best_gmeans_baseline:
            best_gmeans_baseline = gmeans
            torch.save(model.state_dict(), f"{MODELS_DIR}/fraudguard_baseline.pt")

    # MLflow: Log final best metric
    mlflow.log_metric("best_gmeans", best_gmeans_baseline)

# Capture Baseline Metrics
baseline_vram = torch.cuda.max_memory_allocated() / 1e9

model.load_state_dict(torch.load(f"{MODELS_DIR}/fraudguard_baseline.pt"))
model.eval()
latencies_baseline = []
with torch.no_grad():
    for batch in val_loader:
        batch = batch.to(device)
        start = time.perf_counter()
        _ = model(batch.x, batch.edge_index)
        latencies_baseline.append((time.perf_counter() - start) * 1000)

baseline_p95 = np.percentile(latencies_baseline, 95)

print(f"\n✅ Baseline VRAM: {baseline_vram:.2f} GB")
print(f"✅ Baseline P95 Latency: {baseline_p95:.2f} ms")
print(f"🏁 Baseline Best G-Means: {best_gmeans_baseline*100:.2f}%")

# Clean up
del model, trainer, optimized_data, train_loader, val_loader
torch.cuda.empty_cache()
gc.collect()

print(f"--- Baseline Training took {(time.time() - start_time):.2f} seconds ---")

Initializing Vanilla Baseline (No MCD, No RL)...


/usr/local/lib/python3.12/dist-packages/torch_geometric/loader/neighbor_loader.py:229: UserWarning: Using 'NeighborSampler' without a 'pyg-lib' installation is deprecated and will be removed soon. Please install 'pyg-lib' for accelerated neighborhood sampling
  neighbor_sampler = NeighborSampler(



 Starting Baseline Training (30 Epochs)...
Baseline Epoch  1 | Spec: 85.23% | Recall: 16.16% | G-Means: 37.11%
Baseline Epoch  2 | Spec: 85.61% | Recall: 16.92% | G-Means: 38.05%
Baseline Epoch  3 | Spec: 78.60% | Recall: 25.42% | G-Means: 44.70%
Baseline Epoch  4 | Spec: 76.83% | Recall: 27.26% | G-Means: 45.76%
Baseline Epoch  5 | Spec: 87.65% | Recall: 17.05% | G-Means: 38.65%
Baseline Epoch  6 | Spec: 86.29% | Recall: 17.96% | G-Means: 39.36%
Baseline Epoch  7 | Spec: 92.88% | Recall: 11.84% | G-Means: 33.16%
Baseline Epoch  8 | Spec: 84.39% | Recall: 21.51% | G-Means: 42.61%
Baseline Epoch  9 | Spec: 92.79% | Recall: 11.36% | G-Means: 32.47%
Baseline Epoch 10 | Spec: 88.59% | Recall: 17.94% | G-Means: 39.86%
Baseline Epoch 11 | Spec: 93.22% | Recall: 12.54% | G-Means: 34.18%
Baseline Epoch 12 | Spec: 87.18% | Recall: 19.97% | G-Means: 41.73%
Baseline Epoch 13 | Spec: 86.93% | Recall: 19.82% | G-Means: 41.51%
Baseline Epoch 14 | Spec: 90.78% | Recall: 15.83% | G-Means: 37.91%
Base

# Improved AD-RL-GNN

In [6]:
start_time = time.time()

model_cfg = load_model_config()
data_cfg = load_data_config()
data_cfg.paths.raw_data_dir = Path(DATA_DIR)

# Aligned Hyperparameters (Strict Ceteris Paribus)
model_cfg.training["max_epochs"] = MAX_EPOCHS
model_cfg.training["learning_rate"] = LEARNING_RATE
model_cfg.rl_agent["reward_scaling"] = 2.0
model_cfg.adaptive_mcd["alpha"] = GOLD_ALPHA  # Aggressive cleaning
model_cfg.graph.similarity_threshold = 0.75

print(f"\n🚀 Initializing AD-RL (MCD=ON, RL=ON)...")
trainer = FraudTrainer(model_config=model_cfg, data_config=data_cfg, device=device)

# Reset Stats
torch.cuda.reset_peak_memory_stats()

# Re-process
trainer._preprocess(train_df, val_df, test_df)
trainer._build_graph()
trainer._prepare_labels(train_df, val_df, test_df)

# Save preprocessor artifacts for API deployment (from the deployed model)
import pickle
PROCESSED_DIR = f"{MODELS_DIR}/processed"
os.makedirs(PROCESSED_DIR, exist_ok=True)

with open(f"{PROCESSED_DIR}/scaler.pkl", "wb") as f:
    pickle.dump(trainer.preprocessor.scaler, f)

with open(f"{PROCESSED_DIR}/pca.pkl", "wb") as f:
    pickle.dump(trainer.preprocessor.pca, f)

print(f"✅ Preprocessor artifacts saved to {PROCESSED_DIR}")

print("\n🧠 Training AdaptiveMCD (Alpha 0.80)...")
trainer._train_mcd()

print("\n🤖 Training RL Agent...")
trainer._train_rl_and_enhance()

# VRAM Flush
print("\n🧹 Flushing VRAM before GNN Training...")
torch.cuda.empty_cache()
gc.collect()

# Apply class weight
weights = torch.tensor([1.0, FRAUD_WEIGHT]).to(device)
trainer._init_model()
trainer.criterion = torch.nn.CrossEntropyLoss(weight=weights)
model = trainer.model
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-4)

# Loaders
optimized_data = Data(x=trainer.X_full, edge_index=trainer.edge_index, y=trainer.all_labels)
optimized_data.train_mask = trainer.train_mask
optimized_data.val_mask = trainer.val_mask
train_loader = NeighborLoader(optimized_data, num_neighbors=NUM_NEIGHBORS, batch_size=BATCH_SIZE, input_nodes=optimized_data.train_mask, shuffle=True)
val_loader = NeighborLoader(optimized_data, num_neighbors=NUM_NEIGHBORS, batch_size=BATCH_SIZE, input_nodes=optimized_data.val_mask, shuffle=False)

print(f"\n🚀 Starting AD-RL Training ({MAX_EPOCHS} Epochs)...")
best_gmeans_gold = 0

# MLflow: Start AD-RL-GNN run
with mlflow.start_run(run_name="AD-RL-GNN"):
    mlflow.log_params({
        "model_type": "AD-RL-GNN",
        "alpha": GOLD_ALPHA,
        "fraud_weight": FRAUD_WEIGHT,
        "learning_rate": LEARNING_RATE,
        "batch_size": BATCH_SIZE,
        "max_epochs": MAX_EPOCHS,
        "num_neighbors": str(NUM_NEIGHBORS),
        "gradient_clip": GRADIENT_CLIP,
        "reward_scaling": 2.0
    })

    for epoch in range(MAX_EPOCHS):
        model.train()
        for batch in train_loader:
            batch = batch.to(device)
            optimizer.zero_grad()
            out = model(batch.x, batch.edge_index)
            loss = trainer.criterion(out[:batch.batch_size], batch.y[:batch.batch_size])

            # Gradient clipping
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=GRADIENT_CLIP)
            optimizer.step()

        # Eval every epoch
        model.eval()
        all_preds, all_true = [], []
        with torch.no_grad():
            for batch in val_loader:
                batch = batch.to(device)
                out = model(batch.x, batch.edge_index)
                pred = out[:batch.batch_size].argmax(dim=1)
                all_preds.extend(pred.cpu().numpy())
                all_true.extend(batch.y[:batch.batch_size].cpu().numpy())

        metrics = evaluator.compute_metrics(np.array(all_true), np.array(all_preds))
        gmeans = metrics['gmeans']

        # MLflow: Log metrics per epoch
        mlflow.log_metrics({
            "gmeans": gmeans,
            "recall": metrics['recall'],
            "specificity": metrics['specificity']
        }, step=epoch)

        print(f"Epoch {epoch+1:>3} | Spec: {metrics['specificity']*100:.2f}% | Recall: {metrics['recall']*100:.2f}% | G-Means: {gmeans*100:.2f}%")

        if gmeans > best_gmeans_gold:
            best_gmeans_gold = gmeans
            torch.save(model.state_dict(), f"{MODELS_DIR}/fraudguard_AD_RL.pt")

    # MLflow: Log final metrics and model
    mlflow.log_metric("best_gmeans", best_gmeans_gold)

    # Load best model and log to MLflow
    model.load_state_dict(torch.load(f"{MODELS_DIR}/fraudguard_AD_RL.pt"))
    mlflow.pytorch.log_model(model, "fraud_gnn", registered_model_name="FraudGuard-Production")
    print("\n📊 Model logged to MLflow Model Registry")

# Capture AD-RL Metrics
gold_vram = torch.cuda.max_memory_allocated() / 1e9

model.eval()
latencies_gold = []
with torch.no_grad():
    for batch in val_loader:
        batch = batch.to(device)
        start = time.perf_counter()
        _ = model(batch.x, batch.edge_index)
        latencies_gold.append((time.perf_counter() - start) * 1000)

gold_p95 = np.percentile(latencies_gold, 95)

print(f"\n✅ Gold VRAM: {gold_vram:.2f} GB")
print(f"✅ Gold P95 Latency: {gold_p95:.2f} ms")
print(f"🏁 Final Best G-Means: {best_gmeans_gold*100:.2f}%")

# Clean up
del model, trainer, optimized_data, train_loader, val_loader
torch.cuda.empty_cache()
gc.collect()

print(f"--- AD-RL Training took {(time.time() - start_time):.2f} seconds ---")


🚀 Initializing AD-RL (MCD=ON, RL=ON)...
✅ Preprocessor artifacts saved to /content/drive/MyDrive/fraudguard-models/processed

🧠 Training AdaptiveMCD (Alpha 0.80)...

🤖 Training RL Agent...

🧹 Flushing VRAM before GNN Training...


/usr/local/lib/python3.12/dist-packages/torch_geometric/loader/neighbor_loader.py:229: UserWarning: Using 'NeighborSampler' without a 'pyg-lib' installation is deprecated and will be removed soon. Please install 'pyg-lib' for accelerated neighborhood sampling
  neighbor_sampler = NeighborSampler(



🚀 Starting AD-RL Training (30 Epochs)...
Epoch   1 | Spec: 25.15% | Recall: 77.03% | G-Means: 44.02%
Epoch   2 | Spec: 5.73% | Recall: 95.36% | G-Means: 23.38%
Epoch   3 | Spec: 24.22% | Recall: 79.18% | G-Means: 43.79%
Epoch   4 | Spec: 19.94% | Recall: 84.71% | G-Means: 41.10%
Epoch   5 | Spec: 21.20% | Recall: 83.58% | G-Means: 42.10%
Epoch   6 | Spec: 38.67% | Recall: 69.07% | G-Means: 51.68%
Epoch   7 | Spec: 44.97% | Recall: 63.33% | G-Means: 53.37%
Epoch   8 | Spec: 40.46% | Recall: 65.80% | G-Means: 51.60%
Epoch   9 | Spec: 49.62% | Recall: 59.01% | G-Means: 54.11%
Epoch  10 | Spec: 37.89% | Recall: 70.44% | G-Means: 51.66%
Epoch  11 | Spec: 38.59% | Recall: 68.58% | G-Means: 51.44%
Epoch  12 | Spec: 39.24% | Recall: 70.03% | G-Means: 52.42%
Epoch  13 | Spec: 55.48% | Recall: 57.32% | G-Means: 56.39%
Epoch  14 | Spec: 42.78% | Recall: 66.49% | G-Means: 53.34%
Epoch  15 | Spec: 49.20% | Recall: 62.07% | G-Means: 55.26%
Epoch  16 | Spec: 50.25% | Recall: 59.51% | G-Means: 54.68%

2026/01/22 17:35:14 WARNING mlflow.utils.requirements_utils: Found torch version (2.9.0+cu126) contains a local version label (+cu126). MLflow logged a pip requirement for this package as 'torch==2.9.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.


Epoch  30 | Spec: 54.77% | Recall: 59.49% | G-Means: 57.08%


2026/01/22 17:35:24 WARNING mlflow.utils.requirements_utils: Found torch version (2.9.0+cu126) contains a local version label (+cu126). MLflow logged a pip requirement for this package as 'torch==2.9.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2026/01/22 17:35:24 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'FraudGuard-Production'.
Created version '1' of model 'FraudGuard-Production'.



📊 Model logged to MLflow Model Registry

✅ Gold VRAM: 10.62 GB
✅ Gold P95 Latency: 27.84 ms
🏁 Final Best G-Means: 57.21%
--- AD-RL Training took 8495.12 seconds ---


## 6️⃣ Evaluation & Claims Verification

In [7]:
# Compute improvement using Evaluator method
gmeans_improvement = evaluator.compute_gmeans_improvement(best_gmeans_baseline, best_gmeans_gold)

print("\n" + "="*60)
print("🎯 FINAL ARCHITECTURAL COMPARISON (Scientifically Aligned)")
print("="*60)
print(f"| Metric      | Baseline | Optimized | Improvement |")
print(f"|-------------|----------|-----------|-------------|")
print(f"| G-Means     | {best_gmeans_baseline*100:.1f}%    | {best_gmeans_gold*100:.1f}%     | +{gmeans_improvement:.1f}%        |")
print(f"| P95 Latency | {baseline_p95:.1f} ms  | {gold_p95:.1f} ms   | {((baseline_p95-gold_p95)/baseline_p95)*100:.1f}%         |")
print(f"| Peak VRAM   | {baseline_vram:.1f} GB   | {gold_vram:.1f} GB    | {((baseline_vram-gold_vram)/baseline_vram)*100:.1f}%         |")
print("="*60)


🎯 FINAL ARCHITECTURAL COMPARISON (Scientifically Aligned)
| Metric      | Baseline | Optimized | Improvement |
|-------------|----------|-----------|-------------|
| G-Means     | 46.6%    | 57.2%     | +22.7%        |
| P95 Latency | 26.9 ms  | 27.8 ms   | -3.7%         |
| Peak VRAM   | 3.0 GB   | 10.6 GB    | -256.1%         |


In [8]:
from google.colab import runtime
runtime.unassign()